In [3]:
words = open('Data\\names.txt', 'r').read().splitlines()
len(words)


32033

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [302]:
import torch
N = torch.zeros((27, 27), dtype=torch.int32)


In [303]:

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [20]:
P = N[0].float()
P/P.sum()


tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [278]:
# g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(P, num_samples=1, replacement=True).item()
itos[ix]


'n'

In [304]:
P = (N+1).float()
P /=P.sum(1, keepdim=True)
P.shape

torch.Size([27, 27])

In [289]:
P.sum(1, keepdim=True)

tensor([[32033],
        [33885],
        [ 2645],
        [ 3532],
        [ 5496],
        [20423],
        [  905],
        [ 1927],
        [ 7616],
        [17701],
        [ 2900],
        [ 5040],
        [13958],
        [ 6642],
        [18327],
        [ 7934],
        [ 1026],
        [  272],
        [12700],
        [ 8106],
        [ 5570],
        [ 3135],
        [ 2573],
        [  929],
        [  697],
        [ 9776],
        [ 2398]])

In [323]:
g = torch.Generator().manual_seed(2147483647)

for i in range(7):
    ix = 0
    out=[]
    while True:    
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix ==0:
            break
    print(''.join(out))


junide.
janasah.
p.
cony.
a.
nn.
kohin.


In [325]:
g.get_state()

torch.Size([5056])

In [364]:
g = torch.Generator().manual_seed(2147483647)
x = torch.randint(4,(3,2), generator=g)
print(x)

y = torch.randint(4,(3,2))
print(y)

tensor([[3, 2],
        [1, 0],
        [1, 0]])
tensor([[1, 3],
        [3, 3],
        [1, 2]])


Summary
torch.Generator().manual_seed(seed):

Creates an instance-specific generator with its own seed.
Allows for fine-grained control over multiple independent random number sequences.
torch.manual_seed(seed):

Sets the seed for the global random number generator.
Ensures reproducibility for all random operations in PyTorch when no specific generator is provided.

In [365]:
# Set the global seed
torch.manual_seed(2147483647)

# Sample using the global random number generator
print("Global RNG samples:")
print(torch.randn(3))
print(torch.randn(3))

# Create a specific generator with its own seed
g = torch.Generator().manual_seed(1234)

# Sample using the specific generator
print("\nSpecific generator samples:")
print(torch.randn(3, generator=g))
print(torch.randn(3, generator=g))

# Reset the global seed and sample again
torch.manual_seed(2147483647)
print("\nGlobal RNG samples after reset:")
print(torch.randn(3))
print(torch.randn(3))

Global RNG samples:
tensor([-0.9800, -1.6578, -0.0572])
tensor([-0.3409,  2.0992,  0.8964])

Specific generator samples:
tensor([ 0.0461,  0.4024, -1.0115])
tensor([ 0.2167, -0.6123,  0.5036])

Global RNG samples after reset:
tensor([-0.9800, -1.6578, -0.0572])
tensor([-0.3409,  2.0992,  0.8964])


In [395]:

log_likelihood = 0.0
n = 0

for w in words:
#for w in ["andrejq"]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    # print(f'{prob.item():.4f}')
    # logprob = torch.log(prob)
    logprob = torch.log(torch.tensor(1/27))
    log_likelihood+=logprob
    n+=1
print(f'This is the LOGLikley {-log_likelihood:.4f}')
print(f'This is the AVG Negative LOGLikley {-log_likelihood/n:.4f}')



This is the LOGLikley 751979.9375
This is the AVG Negative LOGLikley 3.2960


In [409]:
xs = []
ys=[]
for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    print(f'{ch1}, {ch2}')
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

., e
e, m
m, m
m, a
a, .


In [413]:
xenc = torch.nn.functional.one_hot(xs, num_classes=27).float()

In [418]:
xenc.shape

torch.Size([5, 27])

In [427]:
torch.manual_seed(42)
w = torch.randn((27,27))
logits = xenc@w
logits = logits.exp()
probs = logits/logits.sum(1, keepdim=True)
probs

tensor([[0.1230, 0.0793, 0.0441, 0.0022, 0.0353, 0.0052, 0.0172, 0.0036, 0.0084,
         0.0932, 0.0121, 0.0044, 0.0087, 0.0102, 0.0083, 0.0384, 0.0926, 0.0153,
         0.0109, 0.0278, 0.0084, 0.0527, 0.0399, 0.0962, 0.0644, 0.0655, 0.0330],
        [0.0396, 0.0698, 0.0227, 0.0037, 0.0562, 0.0153, 0.0626, 0.0112, 0.0421,
         0.0084, 0.0581, 0.1125, 0.0951, 0.0292, 0.0181, 0.0107, 0.0247, 0.0316,
         0.0586, 0.0140, 0.1109, 0.0019, 0.0147, 0.0067, 0.0502, 0.0033, 0.0280],
        [0.0123, 0.0078, 0.0250, 0.0187, 0.0050, 0.0125, 0.0074, 0.0099, 0.1254,
         0.1147, 0.0934, 0.0176, 0.0144, 0.0029, 0.0085, 0.0116, 0.1132, 0.0138,
         0.0203, 0.1464, 0.0943, 0.0242, 0.0326, 0.0140, 0.0340, 0.0149, 0.0053],
        [0.0123, 0.0078, 0.0250, 0.0187, 0.0050, 0.0125, 0.0074, 0.0099, 0.1254,
         0.1147, 0.0934, 0.0176, 0.0144, 0.0029, 0.0085, 0.0116, 0.1132, 0.0138,
         0.0203, 0.1464, 0.0943, 0.0242, 0.0326, 0.0140, 0.0340, 0.0149, 0.0053],
        [0.0934, 0.0195,

In [433]:
probs[0].sum()

tensor(1.0000)

In [444]:
nll = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------', x)
  p = probs[i,y]
  nll[i] = -torch.log(p).item()
  print(nll[i])
nll.sum()/5


-------- 0
tensor(5.2568)
-------- 5
tensor(3.5344)
-------- 13
tensor(5.8390)
-------- 13
tensor(4.8595)
-------- 1
tensor(2.3710)


tensor(4.3721)

In [446]:
nll.mean().item()

4.372129440307617

Now FINALLY DO ALL

In [513]:
xs = []
ys=[]
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    # print(f'{ch1}, {ch2}')
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.nelement()
xenc = torch.nn.functional.one_hot(xs, num_classes=27).float()


In [514]:
xenc.shape


torch.Size([228146, 27])

In [507]:
ys.shape

torch.Size([228146])

In [510]:
torch.manual_seed(42)
W = torch.randn((27,27), requires_grad=True)

for i in range (1000):
    logits = xenc@W
    logits = logits.exp()
    probs = logits/logits.sum(1, keepdim=True)
    # nll = -torch.log(probs[torch.arange(num), y]).mean()
    nll = -probs[torch.arange(num), ys].log().mean()
    print(nll.item())
    W.grad= None
    nll.backward()
    W.data += -10*W.grad


2.4626004695892334
2.462590456008911
2.462580680847168
2.4625706672668457
2.4625608921051025
2.4625508785247803
2.462541341781616
2.462531328201294
2.46252179145813
2.4625117778778076
2.4625020027160645
2.4624924659729004
2.4624826908111572
2.462473154067993
2.46246337890625
2.462453842163086
2.462444305419922
2.462434768676758
2.4624249935150146
2.4624154567718506
2.4624061584472656
2.4623963832855225
2.4623870849609375
2.4623775482177734
2.4623680114746094
2.4623584747314453
2.4623494148254395
2.4623398780822754
2.4623305797576904
2.4623210430145264
2.4623117446899414
2.4623024463653564
2.4622933864593506
2.4622840881347656
2.4622747898101807
2.4622654914855957
2.46225643157959
2.462247133255005
2.46223783493042
2.462228775024414
2.462219715118408
2.4622104167938232
2.4622013568878174
2.4621922969818115
2.4621832370758057
2.4621741771698
2.462165117263794
2.462156057357788
2.4621472358703613
2.4621379375457764
2.4621288776397705
2.462120294570923
2.462111234664917
2.4621024131774902


In [488]:
xenc

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [527]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(7):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    # xenc = torch.nn.functional.one_hot(torch.tensor(ix), num_classes=27).float().unsqueeze(0) 
    # ## this unsqueeze works because 
    #torch.tensor(ix) creates a tensor of rown vector (ndim=1) but the xenc@w matmul requires xenc to be (_,27) and ndim=2 , 
    #torch.tensor([ix]) creates a tensor of 1,27 which is okay to use as is
    xenc = torch.nn.functional.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

RuntimeError: prob_dist must be 1 or 2 dim

In [537]:
l = torch.nn.functional.one_hot(torch.tensor(ix), num_classes=27).float()
# l.unsqueeze(0)
l.ndim
torch.tensor([ix])

tensor([0])